## Problem 7 : Game of Tic-Tac-Toe

In [76]:
import time
import random
import collections
import numpy as np
import matplotlib.pyplot as plt
        
class TicTacToe(object):
    """A grid world with pellets to collect and an enemy to avoid."""

    def __init__(self):
        """Environments have fixed size and pellet counts."""
        self.board = np.zeros((3,3))
        self.isEnd = False
        self.next_reward = 0
    
    def initialise(self):
        self.board = np.zeros((3,3))
        self.isEnd = False
        self.next_reward = 0
        return random.choice([1,-1])

    def actions(self):
        positions = []
        for i in range(3):
            for j in range(3):
                if self.board[i, j] == 0:
                    positions.append((i, j))  # need to be tuple
        return positions

    def reward(self):
        result = self.winner()
        # backpropagate reward
        if result == 1:
            return  10
        elif result == -1:
            return -10
        elif result == 0:
            return 1
        else:
            return 0
    
    def winner(self):
        # row
        for i in range(3):
            if sum(self.board[i, :]) == 3:
                self.isEnd = True
                return 1
            if sum(self.board[i, :]) == -3:
                self.isEnd = True
                return -1
        # col
        for i in range(3):
            if sum(self.board[:, i]) == 3:
                self.isEnd = True
                return 1
            if sum(self.board[:, i]) == -3:
                self.isEnd = True
                return -1
        # diagonal
        diag_sum1 = sum([self.board[i, i] for i in range(3)])
        diag_sum2 = sum([self.board[i, 3 - i - 1] for i in range(3)])
        diag_sum = max(abs(diag_sum1), abs(diag_sum2))
        if diag_sum == 3:
            self.isEnd = True
            if diag_sum1 == 3 or diag_sum2 == 3:
                return 1
            else:
                return -1

        # tie
        # no available positions
        if len(self.actions()) == 0:
            self.isEnd = True
            return 0
        # not end
        self.isEnd = False
        return None

    def act(self, position, symbol):
        self.board[position] = symbol
        

    def state(self):
        return str(self.board.reshape(3*3))

In [77]:
class Agent_Random(object):
    """Learns to act random within the environment."""

    def __init__(self):
        pass

    def policy(self, s, actions):
        return random.choice(actions)

class Agent_Safe(object):
    """Learns to act safe within the environment."""

    def __init__(self):
        pass

    def policy(self, board, actions):
        # row
        for i in range(3):
            if sum(board[i, :]) == 2 or sum(board[i, :]) == -2:
                return (i,list(board[i,:]).index(0))
        # col
        for i in range(3):
            if sum(board[:, i]) == 2 or sum(board[:, i]) == -2:
                return (list(board[:,i]).index(0),i)
            
        # diagonal
        diag1 = [board[i, i] for i in range(3)]
        diag2 = [board[i, 3 - i - 1] for i in range(3)]

        if sum(diag1) == 2 or sum(diag1) == -2:
                return (diag1.index(0),diag1.index(0))
        
        if sum(diag2) == 2 or sum(diag2) == -2:
                return (diag2.index(0),diag2.index(0))
        
        return random.choice(actions)
        
        

class Agent_Qlearn(object):
    """Learns to act within the environment."""

    def __init__(self):
        self.epsilon = 0.05 # Exploration rate
        self.gamma = 0.9 # Discount factor
        self.alpha = 0.01 # Learning rate
        self.Q_values = {}

    def choose(self, s, actions):
        """Return an action to try in this state."""
        p = random.random()
        if p < self.epsilon:
            return random.choice(actions)
        else:
            return self.policy(s, actions)

    def policy(self, s, actions):
        """Return the best action for this state."""
        max_value = max([self.Q(s,a) for a in actions])
        max_actions = [a for a in actions if self.Q(s,a) == max_value]
        return random.choice(max_actions)

    def Q(self, s, a):
        """Return the estimated Q-value of this action in this state."""
        if (s,a) not in self.Q_values:
            self.Q_values[(s,a)] = 0
        return self.Q_values[(s,a)]
    
    def observe(self, s, a, sp, r, actions):
        """Update weights based on this observed step."""
        if len(actions) == 0:
            max_value = 0
        else:
            max_value = max([self.Q(sp,a) for a in actions])
        self.Q_values[(s,a)] = (1-self.alpha)*self.Q(s,a) + self.alpha*(r + self.gamma*max_value)

In [78]:
def test(environment,agent,opponent,rounds=100):
    rewards = []
    for epoch in range(rounds):
        turn = environment.initialise()
        if turn == -1:
            actions = environment.actions()
            a_ = opponent.policy(environment.board,actions)
            environment.act(a_,-1)
        while not environment.isEnd:   
            s = environment.state()
            actions = environment.actions()
            a = agent.policy(s, actions)   
            environment.act(a,1)

            win = environment.winner()
            if win is not None:
                r = environment.reward()
            else:
                actions = environment.actions()
                a_ = opponent.policy(environment.board,actions)
                environment.act(a_,-1)
                r = environment.reward()
        rewards.append(r)
    print('Wins', rewards.count(10),'Loses', rewards.count(-10), 'Draws', rewards.count(1))

def train(environment,agent,opponents):
    for epoch in range(10000):
        opponent = random.choice(opponents)
        turn = environment.initialise()
        if turn == -1:
            actions = environment.actions()
            a_ = opponent.policy(environment.board,actions)
            environment.act(a_,-1)
        while not environment.isEnd:   
            s = environment.state()
            actions = environment.actions()
            a = agent.choose(s, actions)   
            environment.act(a,1)
    #         print(environment.board)
            sp = environment.state()
            win = environment.winner()
            if win is not None:
                r = environment.reward()
            else:
                actions = environment.actions()
                a_ = opponent.policy(environment.board,actions)
                environment.act(a_,-1)
    #             print(environment.board)
                sp = environment.state()
                r = environment.reward()
            actions = environment.actions()
            agent.observe(s,a,sp,r,actions)
        if epoch%200 == 0:
            print('Epoch',epoch,end=' ')
            test(environment,agent,opponent)
    return agent

environment = TicTacToe()
opponent1 = Agent_Random()
agent_q1 = Agent_Qlearn()

opponent2 = Agent_Safe()
agent_q2 = Agent_Qlearn()

agent_q3 = Agent_Qlearn()

In [79]:
#1 Training against random agent
agent_q1 = train(environment,agent_q1,[opponent1])

Epoch 0 Wins 45 Loses 44 Draws 11
Epoch 200 Wins 53 Loses 39 Draws 8
Epoch 400 Wins 56 Loses 35 Draws 9
Epoch 600 Wins 54 Loses 34 Draws 12
Epoch 800 Wins 62 Loses 25 Draws 13
Epoch 1000 Wins 69 Loses 20 Draws 11
Epoch 1200 Wins 67 Loses 25 Draws 8
Epoch 1400 Wins 72 Loses 24 Draws 4
Epoch 1600 Wins 75 Loses 19 Draws 6
Epoch 1800 Wins 71 Loses 20 Draws 9
Epoch 2000 Wins 75 Loses 14 Draws 11
Epoch 2200 Wins 79 Loses 14 Draws 7
Epoch 2400 Wins 85 Loses 10 Draws 5
Epoch 2600 Wins 79 Loses 12 Draws 9
Epoch 2800 Wins 80 Loses 8 Draws 12
Epoch 3000 Wins 82 Loses 8 Draws 10
Epoch 3200 Wins 75 Loses 8 Draws 17
Epoch 3400 Wins 80 Loses 6 Draws 14
Epoch 3600 Wins 81 Loses 7 Draws 12
Epoch 3800 Wins 71 Loses 6 Draws 23
Epoch 4000 Wins 85 Loses 5 Draws 10
Epoch 4200 Wins 79 Loses 5 Draws 16
Epoch 4400 Wins 85 Loses 4 Draws 11
Epoch 4600 Wins 88 Loses 4 Draws 8
Epoch 4800 Wins 86 Loses 4 Draws 10
Epoch 5000 Wins 83 Loses 3 Draws 14
Epoch 5200 Wins 80 Loses 5 Draws 15
Epoch 5400 Wins 76 Loses 3 Draw

In [80]:
#2 Training against safe agent
agent_q2 = train(environment,agent_q2,[opponent2])

Epoch 0 Wins 13 Loses 72 Draws 15
Epoch 200 Wins 11 Loses 66 Draws 23
Epoch 400 Wins 17 Loses 60 Draws 23
Epoch 600 Wins 8 Loses 59 Draws 33
Epoch 800 Wins 27 Loses 39 Draws 34
Epoch 1000 Wins 27 Loses 32 Draws 41
Epoch 1200 Wins 31 Loses 32 Draws 37
Epoch 1400 Wins 29 Loses 29 Draws 42
Epoch 1600 Wins 49 Loses 16 Draws 35
Epoch 1800 Wins 33 Loses 22 Draws 45
Epoch 2000 Wins 45 Loses 14 Draws 41
Epoch 2200 Wins 34 Loses 15 Draws 51
Epoch 2400 Wins 40 Loses 13 Draws 47
Epoch 2600 Wins 35 Loses 11 Draws 54
Epoch 2800 Wins 38 Loses 10 Draws 52
Epoch 3000 Wins 37 Loses 6 Draws 57
Epoch 3200 Wins 45 Loses 6 Draws 49
Epoch 3400 Wins 34 Loses 8 Draws 58
Epoch 3600 Wins 44 Loses 6 Draws 50
Epoch 3800 Wins 38 Loses 1 Draws 61
Epoch 4000 Wins 43 Loses 5 Draws 52
Epoch 4200 Wins 42 Loses 3 Draws 55
Epoch 4400 Wins 42 Loses 6 Draws 52
Epoch 4600 Wins 39 Loses 4 Draws 57
Epoch 4800 Wins 38 Loses 1 Draws 61
Epoch 5000 Wins 34 Loses 5 Draws 61
Epoch 5200 Wins 43 Loses 5 Draws 52
Epoch 5400 Wins 47 Lo

In [82]:
#3 Training against both agent randomly
agent_q3 = train(environment,agent_q3,[opponent1,opponent2])

Epoch 0 Wins 45 Loses 44 Draws 11
Epoch 200 Wins 15 Loses 68 Draws 17
Epoch 400 Wins 47 Loses 37 Draws 16
Epoch 600 Wins 14 Loses 67 Draws 19
Epoch 800 Wins 20 Loses 54 Draws 26
Epoch 1000 Wins 30 Loses 54 Draws 16
Epoch 1200 Wins 44 Loses 43 Draws 13
Epoch 1400 Wins 62 Loses 24 Draws 14
Epoch 1600 Wins 61 Loses 28 Draws 11
Epoch 1800 Wins 61 Loses 27 Draws 12
Epoch 2000 Wins 72 Loses 20 Draws 8
Epoch 2200 Wins 62 Loses 15 Draws 23
Epoch 2400 Wins 53 Loses 19 Draws 28
Epoch 2600 Wins 60 Loses 20 Draws 20
Epoch 2800 Wins 77 Loses 10 Draws 13
Epoch 3000 Wins 74 Loses 11 Draws 15
Epoch 3200 Wins 53 Loses 17 Draws 30
Epoch 3400 Wins 75 Loses 9 Draws 16
Epoch 3600 Wins 60 Loses 6 Draws 34
Epoch 3800 Wins 69 Loses 8 Draws 23
Epoch 4000 Wins 81 Loses 7 Draws 12
Epoch 4200 Wins 69 Loses 6 Draws 25
Epoch 4400 Wins 67 Loses 4 Draws 29
Epoch 4600 Wins 61 Loses 5 Draws 34
Epoch 4800 Wins 68 Loses 7 Draws 25
Epoch 5000 Wins 67 Loses 6 Draws 27
Epoch 5200 Wins 76 Loses 4 Draws 20
Epoch 5400 Wins 75 

In [83]:
# Testing
print('Agent train against random vs random')
test(environment, agent_q1,opponent1,1000)
print('Agent train against random vs safe')
test(environment, agent_q1,opponent2,1000)
print('Agent train against safe vs random')
test(environment, agent_q2,opponent1,1000)
print('Agent train against safe vs safe')
test(environment, agent_q2,opponent2,1000)
print('Agent train against both vs random')
test(environment, agent_q3,opponent1,1000)
print('Agent train against both vs safe')
test(environment, agent_q3,opponent2,1000)

Agent train against random vs random
Wins 845 Loses 24 Draws 131
Agent train against random vs safe
Wins 397 Loses 175 Draws 428
Agent train against safe vs random
Wins 531 Loses 229 Draws 240
Agent train against safe vs safe
Wins 449 Loses 18 Draws 533
Agent train against both vs random
Wins 831 Loses 32 Draws 137
Agent train against both vs safe
Wins 632 Loses 27 Draws 341


(d) Among the three, the third agent which was trained against both performs relatively best. As it now is trained with safe moves and random moves too. More like Ensembling

(e) No the agents are not unbeatable to aany opponent. The training can be improved by optimizing hyperparameters- Training iterations or Training the robot against itself.